<a href="https://colab.research.google.com/github/XG11/ECE420FinalProject/blob/main/ECE420FinalProjectPythonPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, gaussian_gradient_magnitude
from scipy.spatial.distance import euclidean
from scipy.signal import convolve2d
import matplotlib.pyplot as plt

In [ ]:
#applies gaussian kernel in spacial domain to image using sigma blur operator to generate blurred image in each scale
def BlurredImage(image, sigma):
  gaussian = np.zeros((image.shape[0], image.shape[1]))
  for i in range(0, image.shape[0]):
    for j in range(0, image.shape[1]):
      gaussian[i][j] = 1/(2 * np.pi * sigma * sigma) * np.exp(-1 * (np.square(i) + np.square(j))/(2 * np.square(sigma)))
  L = convolve2d(image, gaussian, mode='same')
  return L

#takes in image, sigma, number of scales, and number of octaves
#returns a list of lists with each list containing images with different scales and each list representing octave
def octaves(image, sigma, scale, octave):
  l_images = []
  num_octave = []
  im_copy = image.copy()
  for o in range(0, octave):
    if(o != 0):
      im_copy = cv2.resize(im_copy, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    r_image = im_copy.copy()
    for s in range(0, scale):
       r_image = BlurredImage(r_image, sigma)
       num_octave.append(r_image)
    l_images.append(num_octave)
    for s in range(0, scale):
      num_octave.pop()
  return l_images

#returns difference of gaussian images within each octave, need to call previous octave function to input into this
def DOG(l_images):
  d_images = []
  num_octave = []
  for i in range(0, len(l_images))
    for j in range(0, len(l_images[i]) - 1):
      difference = l_images[i][j+1] - l_images[i][j]
      num_octave.append(difference)
    d_images.append(num_octave)
    num_octave.clear()
  return d_images


l_images = octaves(image, 1, 5, 4) #sigma = 1, scale = 5, octave = 4
d_images = DOG(l_images) #get difference of gaussians

In [ ]:
#this function returns all keypoints taking in the difference of gaussians
def keypoints(d_images, threshold = 0.3):
  keypoints = []
  scale_factor = 2
  for o, octave in enumerate(d_images):
    for s, scale in octave:
      current = scale
      prev = octave[s - 1] if s > 0 else None  # Previous scale (handle the first scale)
      next = octave[s + 1] if s < len(octave) - 1 else None  # Next scale (handle the last scale)
      for i in range(1, current.shape[0] - 1):
        for j in range(1, current.shape[1] - 1):
          current_pixel = current[i][j]
          if abs(current_pixel) < threshold:
            continue

          neighborhood = [prev[i-1:i+2, j-1:j+2], current[i-1:i+2, j-1:j+2], next[i-1:i+2, j-1:j+2]] if prev is not None and next is not None else []
          max_val = np.max(neighborhood)
          min_val = np.min(neighborhood)
          if current_pixel == max_val or current_pixel == min_val:
            original_i = i * (scale_factor ** (o + 1))
            original_j = j * (scale_factor ** (o + 1))
            keypoints.append([o, s, original_i, original_j])

  return keypoints

keypoint_result = keypoints(d_images, threshold = 0.3) #get keypoint results

In [ ]:
#do gradient calculations for neighborhood around image based on scale(sigma level of blurring in particular image
#does calculation of gradients for one keypoint, must be called with each keypoint
def M_theta_calculation(image, keypoint):
    x, y = keypoint[2], keypoint[3]
    scale = keypoint[1]
    if(scale > 2):
      radius = 16
    else:
      radius = 8
    gradient_magnitudes = []
    gradient_orientations = []

    # Compute gradient magnitudes and orientations around the keypoint
    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):
            if (0 < x + i < image.shape[0] - 1) and (0 < y + j < image.shape[1] - 1):
                dx = image[x + i, y + j + 1] - image[x + i, y + j - 1]
                dy = image[x + i + 1, y + j] - image[x + i - 1, y + j]
                magnitude = np.sqrt(dx**2 + dy**2)
                orientation = np.degrees(np.arctan2(dy, dx)) % 360
                gradient_magnitudes.append(magnitude)
                gradient_orientations.append(orientation)
    return gradient_magnitudes, gradient_orientations

#creates histogram based on bin size and returns it and the dominant orientation
def histogram(gradient_magnitudes, gradient_orientations, bin_size):
   orientation_histogram = np.zeros(bin_size)
   for magnitude, orientation in zip(gradient_magnitudes, gradient_orientations):
     if(orientation == 360):
       bin_index = 0
     else:
       bin_index = int(orientation // (360 // bin_size))
     orientation_histogram[bin_index] += magnitude
   dominant_orientation = np.argmax(orientation_histogram) * (360 // bin_size)
   return(orientation_histogram, dominant_orientation)

IndentationError: unexpected indent (<ipython-input-1-267be712619f>, line 6)

In [ ]:
def compute_gradient(image, x, y):
    """
    Compute gradient magnitude and orientation at pixel (x, y) using central difference.
    """
    dx = image[x, y + 1] - image[x, y - 1]  # Central difference in the x direction
    dy = image[x + 1, y] - image[x - 1, y]  # Central difference in the y direction

    magnitude = np.sqrt(dx**2 + dy**2)  # Gradient magnitude
    orientation = np.degrees(np.arctan2(dy, dx)) % 360  # Gradient orientation (0 to 360 degrees)

    return magnitude, orientation

def compute_keypoint_descriptor(image, keypoint, dominant_orientation):
    """
    Compute the descriptor for the keypoint in the image.
    """
    # Extract a scaled window around the keypoint (taking scale into account)
    #scaled_window = extract_scaled_patch(image, keypoint, window_size)

    # Compute gradients and orientations
    #gx, gy = compute_gradients(scaled_window)
    #magnitude, orientation = compute_magnitude_orientation(gx, gy)

    # Create histograms for each 4x4 sub-block
    descriptor = []
    x_start = keypoint[2] - 8
    y_start = keypoint[3] - 8
    for o_x in range(x_start, x_start + 16, 4):
      for o_y in range(y_start, y_start + 16, 4):
        for s_x in range(o_x, o_x + 4):
          for s_y in range(o_y, o_y + 4):
               if s_x > 0 and s_y > 0 and s_x < image.shape[0] - 1 and s_y < image.shape[1] - 1:
                    # Compute the gradient for each pixel (x, y)
                    magnitude, orientation = compute_gradient(image, x, y)
                    # Append the magnitude and orientation for later use

                    #subtract dominant orientation for rotation independance
                    orientation = (orientation - dominant_orientation)%360
                    gradient_magnitudes.append(magnitude)
                    gradient_orientations.append(orientation)
                    desc_values = histogram(gradient_magnitudes, gradient_orientations, 8)
                    for i in range(0, len(desc_values[0])):
                      if(desc_values[0][i] > 0.2):
                        descriptor.append(0.2)
                      else:
                        descriptor.append(desc_values[0][i])
                    gradient_magnitudes.clear()
                    gradient_orientations.clear()
    # Calculate the L2 norm (Euclidean norm) of the descriptor vector
    norm = np.linalg.norm(descriptor)
    # If the norm is not zero, normalize the descriptor (avoid division by zero)
    if norm != 0:
        descriptor_vector /= norm
    return descriptor

